# 1- import dependencies

In [ ]:
# Pytorch- Install CUDA acceleraton version of pytorch

!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio===0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

In [1]:
import gym
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_atari_env
import os 

C:\Users\leilaa\Anaconda3\python.exe: Error while finding module specification for 'atari_py.import.\\Roms\\ROMS' (ModuleNotFoundError: No module named 'atari_py.import')


# 2- test environment

In [ ]:
# www.atarimania.com/roms/Roms.rar

In [2]:
!python -m atari_py.import_roms .\ROMS\ROMS

copying adventure.bin from .\ROMS\ROMS\Adventure (1980) (Atari, Warren Robinett) (CX2613, CX2613P) (PAL).bin to C:\Users\leilaa\Anaconda3\lib\site-packages\atari_py\atari_roms\adventure.bin
copying air_raid.bin from .\ROMS\ROMS\Air Raid (Men-A-Vision) (PAL) ~.bin to C:\Users\leilaa\Anaconda3\lib\site-packages\atari_py\atari_roms\air_raid.bin
copying alien.bin from .\ROMS\ROMS\Alien (1982) (20th Century Fox Video Games, Douglas 'Dallas North' Neubauer) (11006) ~.bin to C:\Users\leilaa\Anaconda3\lib\site-packages\atari_py\atari_roms\alien.bin
copying amidar.bin from .\ROMS\ROMS\Amidar (1982) (Parker Brothers, Ed Temple) (PB5310) ~.bin to C:\Users\leilaa\Anaconda3\lib\site-packages\atari_py\atari_roms\amidar.bin
copying assault.bin from .\ROMS\ROMS\Assault (AKA Sky Alien) (1983) (Bomb - Onbase) (CA281).bin to C:\Users\leilaa\Anaconda3\lib\site-packages\atari_py\atari_roms\assault.bin
copying asterix.bin from .\ROMS\ROMS\Asterix (AKA Taz) (07-27-1983) (Atari, Jerome Domurat, Steve Woita) (

In [3]:
environment_name = 'Breakout-v0'
env = gym.make(environment_name)

In [4]:
env.reset()

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [5]:
env.action_space
# We have 4 discrete action

Discrete(4)

In [6]:
env.observation_space
# observation is a box with the value is ranging from 0 to 255, and the dimention is hight=210, width=160 , 3 => it is an image based model

Box(0, 255, (210, 160, 3), uint8)

In [7]:
# To deploy the model, we have to observe the environment and take action based on observation, not only sample action. => change action to observation
episodes = 5
for episode in range (1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:4.0
Episode:2 Score:2.0
Episode:3 Score:1.0
Episode:4 Score:0.0
Episode:5 Score:0.0


In [8]:
env.close()

# 3-vectorise environment and train model

In [12]:
# we are gonna be train 4 environments at ones
# make_atari_env is a helper from stable baselines that helps create wrapped atari environments.
# allows you to stack the environments together.
env = make_atari_env('Breakout-v0', n_envs=4, seed=0)
env = VecFrameStack(env, n_stack=4)

In [10]:
# env.reset()
# env.render()
# env.close()

In [13]:
log_path = os.path.join("Training", "Logs")
model = A2C('CnnPolicy', env, verbose=1, tensorboard_log=log_path)

# mlpPolicy was great for tabular data or tabular observations, but because our observation are image in this case,
# cnnPolicy is faster to train

C:\Users\leilaa\Anaconda3\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 8000). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:115.)
  return torch._C._cuda_getDeviceCount() > 0


Using cpu device
Wrapping the env in a VecTransposeImage.


In [16]:
# Training the model, for better performance we can set for one or two million
model.learn(total_timesteps=10000)

Logging to Training\Logs\A2C_3
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 320      |
|    ep_rew_mean        | 2.39     |
| time/                 |          |
|    fps                | 27       |
|    iterations         | 100      |
|    time_elapsed       | 74       |
|    total_timesteps    | 2000     |
| train/                |          |
|    entropy_loss       | -1.24    |
|    explained_variance | 0.961    |
|    learning_rate      | 0.0007   |
|    n_updates          | 492      |
|    policy_loss        | -0.056   |
|    value_loss         | 0.0196   |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 332      |
|    ep_rew_mean        | 2.57     |
| time/                 |          |
|    fps                | 26       |
|    iterations         | 200      |
|    time_elapsed       | 149      |
|    total_timesteps    | 4000     |
| train

# 4- save and reload model

In [ ]:
A2C_path = os.path.join('Training', 'Saved Models', 'A2C_Breakout_Model')
model.save(A2C_path)

In [ ]:
del model

In [ ]:
model = A2C.load(A2C_path, env)

# 5-evaluate and test

In [17]:
# evaluate can only perform on one environment. Here we had 4 environment and vectorized them
env = make_atari_env('Breakout-v0', n_envs=1, seed=0)
env = VecFrameStack(env, n_stack=4)

In [9]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)
# (average score value, standard deviation)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\leilaa\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-00b089ca3b64>", line 1, in <module>
    evaluate_policy(model, env, n_eval_episodes=10, render=True)
NameError: name 'model' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\leilaa\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2040, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\leilaa\Anaconda3\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "C:\Users\lei

NameError: name 'model' is not defined

In [11]:
env.close()

In [ ]:
https://www.youtube.com/watch?v=Mut_u40Sqz4